In [4]:
import torch
# from googlenet import GoogLeNet
import torch.nn as nn

# model = GoogLeNet()
# a = torch.load('googlenet_places.extract.pth')

In [5]:
from torchvision.models import mobilenet_v3_small

net = mobilenet_v3_small(pretrained=False)

In [9]:
backbone = nn.Sequential(*list(net.children())[:-2])
backbone

Sequential(
  (0): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU(inplace=True)
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (2): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_s

In [11]:
pseudo_input = torch.randn(32,3,224,224)
out = backbone(pseudo_input)
print(out.shape)

torch.Size([32, 576, 7, 7])


In [6]:
import pandas as pd
import os

raw_df =pd.read_csv('dataset/raw/1112label.csv')
df_reduced = raw_df.iloc[::30, :]
df_reduced

,x,y,img
0,2.133498,0.343483,frame00151
30,2.121173,0.343660,frame00181
60,2.099536,0.344016,frame00211
90,2.074499,0.346534,frame00241
120,2.046836,0.355282,frame00271
...,...,...,...
10260,2.278788,-2.533617,frame10411
10290,2.314519,-2.517358,frame10441
10320,2.350577,-2.500899,frame10471
10350,2.386127,-2.484668,frame10501


In [7]:
from sklearn.model_selection import train_test_split

indices = df_reduced.index.tolist()

train_idx, test_idx = train_test_split(indices, test_size=0.10, random_state=777)
train_idx.sort()
test_idx.sort()

In [9]:
len(train_idx)

312

In [11]:
train_data = df_reduced.loc[train_idx]
test_data = df_reduced.loc[test_idx]

train_data.to_csv('dataset/training/train.csv', index=False)
test_data.to_csv('dataset/validation/test.csv', index=False)

In [19]:
import numpy as np
from PIL import Image
from glob import glob

In [12]:
import numpy as np
from PIL import Image
import os

# 이미지들이 저장된 디렉토리를 지정합니다.
image_directory = 'dataset/images'

# 이미지 파일 이름 목록을 가져옵니다.
image_files = [f for f in os.listdir(image_directory) if f.endswith('.jpg')]

# 이미지들로부터 채널별 누적 합계와 제곱의 누적 합계를 저장할 배열을 초기화합니다.
sum_of_pixels = np.zeros(3)
sum_of_squares = np.zeros(3)
num_of_pixels = 0

# 이미지들을 순회하면서 채널별 합계를 계산합니다.
for file_name in image_files:
    image_path = os.path.join(image_directory, file_name)
    with Image.open(image_path) as img:
        # img = img.resize((width, height))  # 이미지를 모두 같은 크기로 조정합니다.
        img_array = np.array(img) / 255.0  # 이미지를 [0, 1] 범위로 스케일링합니다.
        # 채널별 합계를 계산합니다.
        sum_of_pixels += img_array.sum(axis=(0, 1))
        sum_of_squares += (img_array ** 2).sum(axis=(0, 1))
        num_of_pixels += img_array.shape[0] * img_array.shape[1]

# 채널별 평균(mean)을 계산합니다.
mean = sum_of_pixels / num_of_pixels

# 채널별 분산(variance)을 계산합니다.
variance = (sum_of_squares / num_of_pixels) - (mean ** 2)

# 채널별 표준편차(standard deviation)을 계산합니다.
stddev = np.sqrt(variance)

print(f"Mean per channel: {mean}")
print(f"Standard deviation per channel: {stddev}")


Mean per channel: [0.49084996 0.4717878  0.4826145 ]
Standard deviation per channel: [0.22393189 0.22104482 0.2238242 ]
